In [1]:
csvfile = "dataV2_cohortV3.csv"

In [2]:
import pandas as pd
import statsmodels.formula.api as smf
import numpy as np

In [3]:
df = pd.read_csv(csvfile)

In [4]:
# Mapping the variables to numeric values
mapping_dict = {
    'CONNECTION_activities_onlinegames_p3m': {
        'Not in the past three months': 0,
        'Less than monthly': 1,
        'Monthly': 2,
        'A few times a month': 3,
        'Weekly': 4,
        'A few times a week': 5,
        'Daily or almost daily': 6
    },
    'CONNECTION_social_time_friends_p7d_grouped': {
        'No time': 0,
        'Less than 1 hour': 1,
        '1 to 4 hours': 2,
        '5 or more hours': 3
    },
    'WELLNESS_malach_pines_burnout_measure_depressed': {
        'Never': 0,
        'Almost never': 1,
        'Rarely': 2,
        'Sometimes': 3,
        'Very Often': 4,
        'Always': 5
    }
}

mapped_df = df.copy()

In [5]:
# Filter mapped_df to only include the desired columns
mapped_df = mapped_df[list(mapping_dict.keys())]

In [6]:
# Map values
for column, mapping in mapping_dict.items():
    if column in mapped_df.columns:
        mapped_df[column] = mapped_df[column].map(mapping)

In [7]:
# Drop NA
mapped_df = mapped_df.dropna()

In [8]:
mapped_df

,CONNECTION_activities_onlinegames_p3m,CONNECTION_social_time_friends_p7d_grouped,WELLNESS_malach_pines_burnout_measure_depressed
0,0.0,3.0,2.0
1,0.0,3.0,1.0
2,0.0,3.0,1.0
3,4.0,2.0,2.0
4,4.0,3.0,1.0
...,...,...,...
844,0.0,2.0,5.0
845,0.0,2.0,4.0
846,0.0,1.0,5.0
847,0.0,3.0,3.0
